In [1]:
# image processing
import cv2

# display image
from matplotlib import pyplot as plt

# mathematical operation
import math

In [2]:
class Image:
	''' - store information about image
	'''

	def __init__( self, image, imageNameStr ):
		self.image = image
		self.rgbImage = cv2.cvtColor( self.image, cv2.COLOR_BGR2RGB )
		self.imageNameStr = imageNameStr

class ImagePlotter:
	''' - used to dynamically create image plot using matplotlib
	'''

	def __init__( self, widthPerImageInch, heightPerImageInInch, numberOfColumn ):
		
		# matplotlib pyplot
		self.plt  = plt

		# image width
		self.widthPerImageInch = widthPerImageInch

		# overall width of figure
		self.overallImageWidthInInch = self.widthPerImageInch * numberOfColumn

		# image height
		self.heightPerImageInInch = heightPerImageInInch

		# all image to plot in this figure
		self.imageObjStorageList = []

		# set number of column in figure
		self.numberOfColumn = numberOfColumn

		# number of row 
		self.numberOfRow = None

		# number of image 
		self.numberOfImage = None

		# figure 
		self.figure = None

	def addImageToPlot( self, image, imageNameStr ):
		''' - add image to be plotted
		'''

		# create image object
		imageObj = Image( image, imageNameStr )

		# store image object
		self.imageObjStorageList.append( imageObj )

	def plotAndShowAllImage( self ):
		''' - construct all image in figure and show them 
		'''

		# get number of image in figure
		self.numberOfImage = len( self.imageObjStorageList )

		# calculate number of row
		self.numberOfRow = math.ceil( self.numberOfImage / self.numberOfColumn )

		# calculate overall plot height in inches
		self.overallImageHeightInInch = self.numberOfRow * self.heightPerImageInInch

		# create figure
		self.figure = self.plt.figure( figsize = ( self.overallImageWidthInInch, self.overallImageHeightInInch ) )

		# loop through image object 
		for imagePosition, imageObj in enumerate( self.imageObjStorageList ):

			# set position of image
			self.figure.add_subplot( self.numberOfRow, self.numberOfColumn, imagePosition + 1 )

			# add image to figure
			self.plt.imshow( imageObj.rgbImage )

			# not show the axis
			self.plt.axis( 'off' )

			# set image title
			self.plt.title( imageObj.imageNameStr )
		
		# show figure
		plt.show()

In [3]:
#################################################################################################################
#
# Helper Function
#
#################################################################################################################

def calculateVectorMagnitude( vectorStartPointTuple, vectorEndPointTuple ):
	''' - calculate the magnitude of the given vector

		ARGS: 
			- vectorStartPointTuple ( Tuple )
			- vectorEndPointTuple ( Tuple )

		RETURN: 
			- vectorMagnitudeFloat ( Float )
	'''

	assert len( vectorStartPointTuple ) == len( vectorEndPointTuple ), ' Start and End point of vector must be the same size but got \
		 start point {}, size {} end point {}, size {} '.format( vectorStartPointTuple, len( vectorStartPointTuple ), vectorEndPointTuple, len( vectorEndPointTuple ) )
	
	# init vector magnitude
	vectorMagnitudeFloat = float()

	# loop through each position in vector's start point
	for pointCompomentPosition in range( len( vectorStartPointTuple ) ):

		# add up value for vector's magnitude 
		vectorMagnitudeFloat += ( vectorEndPointTuple[ pointCompomentPosition ] - vectorStartPointTuple[ pointCompomentPosition ] ) ** 2

	# calculate the vector's magnitude
	return math.sqrt( vectorMagnitudeFloat )


#################################################################################################################
#
# Contour Manipulation
#
#################################################################################################################

class ContourStorage:
	''' - store and manage contour object by its type

		- contour has 3 types:
			- square contour
			- triangle contour
			- circle contour
	'''
	
	def __init__( self, contourList ):

		# all contour storage
		self.allContourList = contourList

		# all contour object storage
		self.allContourObjList = list()

		# convert contour to contour object
		# this is going to use for further processing
		self._createContourObj()

		# contour contour in area range storage
		self.contourObjInAreaRangeList = list()

		# square contour object storage
		self.squareContourObjList = list()
		
		# triangle contour object storage
		self.triangleContourObjList = list()
		
		# circle contour object storage
		self.circleContourObjList = list()

		# outer most square contour object storage
		self.outerMostSquareContourObjList = list()

		# outer most triangle contour object storage
		self.outerMostTriangleContourObjList = list()

		# outer most circle contour object storage
		self.outerMostCircleContourObjList = list()

	def _createContourObj( self ):
		''' - construct contour object from list of contour
		'''

		# loop through each contour
		for contour in self.allContourList:

			# create contour object
			contourObj = Contour( contour )

			# store contour object
			self.allContourObjList.append( contourObj )

	def filterContoursOnlyInAreaRange( self, minArea=0, maxArea=0 ):
		''' - get only contour in area range

			ARGS:
				- contourObjList ( list )
				- minArea ( int )
				- maxArea ( int )

			RETURN:
				- contourObjInAreaRangeList ( list )
		'''

		# loop through each contour object
		for contourObj in self.allContourObjList:

			# this contour is in range
			if minArea <= contourObj.area <= maxArea:
				
				# store it
				self.contourObjInAreaRangeList.append( contourObj )
		return self.contourObjInAreaRangeList

	def isOuterMostContour( self, contourObj, otherContourObjList, epsilonPercent ):
		''' - check to see if this contour object is NOT inside any of the other contour object
		'''

		# approximate polygon of contour object
		approximatedPointList = contourObj.approximatePolygonOfContour( epsilonPercent )
		
		# loop through each other contour obj 
		for otherContourObj in otherContourObjList:

			# loop each point in approximated polygon
			for point in approximatedPointList:

				pointTuple = ( int( point[ 0 ][ 0 ] ), int( point[ 0 ][ 1 ] ) )

				# point is inside other approximated polygon
				if cv2.pointPolygonTest( otherContourObj.contour, pointTuple, False ) > 0:
					
					# this is NOT the outer most contour
					return False

		# this is the outer most contour
		return True

	def isSquareContour( self, contourObj, epsilonPercent, errorPercent ):
		''' - check if this contour object is square shape

			- this contour object is square contour if:
				- its approximated curve has 4 points
				- its bounding box has an aspect ration close to 1
		'''

		# calculate approximation curve of contour
		approximatedPointList = contourObj.approximatePolygonOfContour( epsilonPercent  )

		# contour has polygon approximation point equal to 4
		if len( approximatedPointList ) == 4:

			# calculate approximation polygon bounding box
			( xPosition, yPosition, boundingBoxWidth, boundingBoxHeight ) = cv2.boundingRect( approximatedPointList )
			
			# calculate aspect ratio of bounding box
			boundingBoxAspectRatio = boundingBoxWidth / boundingBoxHeight

			# calculate acceptable error
			acceptableError = errorPercent / 100 

			# this bounding box is square
			if 1 - acceptableError <= boundingBoxAspectRatio <= 1 + acceptableError:
				
				# this is square contour
				return True
			
		# this contour is not square
		return False
	
	def isTriangleContour( self, contourObj, epsilonPercent ):
		''' - check if this is a triangle shape contour

			- this contour is triangle if:
				- it approximation curve has 3 points 
		''' 

		# calculate approximation curve of contour
		approximatedPointList = contourObj.approximatePolygonOfContour( epsilonPercent  )

		# this is triangle shape contour
		if len( approximatedPointList ) == 3:
			return True
		
		# this is not a triangle contour
		return False
	
	def isCircleContour( self, contourObj, epsilonPercent, errorPercent ):
		''' - check if this is a circle contour

			- this contour in circle if:
				- approximate point is more than5
				- bounding box has square aspect ratio
		'''

		# calculate approximation curve of contour
		approximatedPointList = contourObj.approximatePolygonOfContour( epsilonPercent  )

		# contour has polygon approximation point more than 5
		if len( approximatedPointList ) > 5:

			# calculate approximation polygon bounding box
			( xPosition, yPosition, boundingBoxWidth, boundingBoxHeight ) = cv2.boundingRect( approximatedPointList )
			
			# calculate aspect ratio of bounding box
			boundingBoxAspectRatio = boundingBoxWidth / boundingBoxHeight

			# calculate acceptable error
			acceptableError = errorPercent / 100 

			# this bounding box is square
			if 1 - acceptableError <= boundingBoxAspectRatio <= 1 + acceptableError:
				
				# this contour is circle
				return True
			
		# this contour is not circle
		return False
	
	def classifyContourObjByShape( self ):
		''' - classify contour object by its shape

			- we are focusing only for 3 type of contour here:
				- square
				- circle
				- triangle
		'''

		# loop through each contour object in area range
		for contourObj in self.contourObjInAreaRangeList:
			
			# this is circle contour
			if self.isCircleContour( contourObj, 4, 6 ):
			
				# store contour object in circle contour storage
				self.circleContourObjList.append( contourObj )

				# set contour's shape type
				contourObj.shapeTypeStr = 'circle'
				continue

			# this is triangle contour
			if self.isTriangleContour( contourObj, 5 ):

				# store contour object in triangle contour storage
				self.triangleContourObjList.append( contourObj )

				# set contour's shape type
				contourObj.shapeTypeStr = 'triangle'
				continue

			# this is square contour
			if self.isSquareContour( contourObj, 4, 5 ):

				# store contour object in square contour storage
				self.squareContourObjList.append( contourObj )

				# set contour's shape type
				contourObj.shapeTypeStr = 'square'
				continue
	
	def filterOnlyOuterMostContourObj( self, contourObjList, epsilonPercent ):
		''' - filter to get only the outer most contour obj from contour object list

			ARGS: 
				- contourObjList ( list ) --> input contour list
				- epsilonPercent ( int )

			RETURN:
				- outerMostContourObjList ( list )
		'''

		# outer most contour object storage
		outerMostContourObjList = list()

		# loop through each contour object
		for contourObj in contourObjList:

			# this is outer most contour object
			if self.isOuterMostContour( contourObj=contourObj, otherContourObjList=contourObjList, epsilonPercent=epsilonPercent ):

				# calculate center point of contour
				contourObj.calculateCenterPoint()

				# calculate coordinate frame of contour
				contourObj.calculateCoordinateFrame()

				# store it
				outerMostContourObjList.append( contourObj )
		
		return outerMostContourObjList
			
class Contour:
	''' - store information about contour of interest
	'''

	def __init__( self, contour ):

		# all pixel coordinate of this contour
		self.contour = contour

		# store the latest epsilon percent value used 
		self.latestEpsilonPercent = None

		# store percent of epsilon to approximated point list of contour
		self.epsilonPercentToApproximatedPointListDict = dict()

		# contour type
		# contour must be one of the following type
		# - square
		# - circle
		# - triangle
		self.shapeTypeStr = None

		# center point of contour
		self.centerPointTuple = tuple()

		# end point of x axis with respect to center point 
		self.xAxisEndPointTuple = tuple()

		# end point of y axis with respect to center point 
		self.yAxisEndPointTuple = tuple()

		# length of vector ( in pixel ) to draw contour's coordinate frame
		self.vectorLengthInt = 50

	@property
	def area( self ):
		''' - find area of contour

			RETURN: 	
				- areaOfContour ( float )
		'''

		# calculate contour area``
		return cv2.contourArea( self.contour )
	
	def approximatePolygonOfContour( self, epsilonPercent ):
		''' - approximate polygon of contour to be a rougher shape

			- mainly used for shape analysis

			- usually set epsilon parameter to 1 - 5 % of contour's perimeter

			- first this function is going to find if the approximated point exist for selected percent of epsilon
				- if exist --> return value from storage
				- else --> calculate, store and return
		'''

		# store the latest epsilon percent used 
		self.latestEpsilonPercent = epsilonPercent

		# approximated point exist for selected epsilon value
		if epsilonPercent in self.epsilonPercentToApproximatedPointListDict:
			
			# just return the calculated approximated point list
			return self.epsilonPercentToApproximatedPointListDict[ epsilonPercent ]

		# the approximated point list dose NOT exist for selected percent of epsilon, then
		# calculate contour perimeter
		contourPerimeter = cv2.arcLength( self.contour, True )

		# approximate curve of contour
		approximatedPointList = cv2.approxPolyDP( self.contour,  epsilonPercent / 100 * contourPerimeter, True )
		
		# store percent of epsilon to approximated point list
		self.epsilonPercentToApproximatedPointListDict[ epsilonPercent ] = approximatedPointList 
		
		return approximatedPointList
	
	def calculateCenterPoint( self ):
		''' - calculate center point of contour using opencv's moment
		'''

		# calculate moment 
		moment = cv2.moments( self.contour )

		# calculate x-axis of center point
		xComponent = int( moment[ "m10" ] / moment[ "m00" ] )

		# calculate y-axis of center point
		yComponent = int( moment[ "m01" ] / moment[ "m00" ] )

		self.centerPointTuple = ( xComponent, yComponent )

	def calculateCoordinateFrame( self ):
		''' - calculate coordinate frame of contour with respect to OpenCV's world coordinate
		'''

		# this contour is circle contour, then
		if self.shapeTypeStr == 'circle':

			# x-axis component is the same direction with camera's x-axis
			self.xAxisEndPointTuple = ( self.centerPointTuple[ 0 ] + self.vectorLengthInt, self.centerPointTuple[ 1 ] )

			# y-axis component is the same direction with camera's y-axis
			self.yAxisEndPointTuple = ( self.centerPointTuple[ 0 ], self.centerPointTuple[ 1 ] + self.vectorLengthInt )

		# this contour is triangle or square contour, then
		# approximate polygon of contour
		approximatedPolygon = self.approximatePolygonOfContour( self.latestEpsilonPercent )

		# approximate polygon of contour in 1D tuple format for each point
		approximatedPolygon1DList = list()

		# loop through each point in approximated polygon
		for point in approximatedPolygon:

			# convert point into 1D tuple
			pointTuple = ( int( point[ 0 ][ 0 ] ), int( point[ 0 ][ 1 ] ) )

			# store it
			approximatedPolygon1DList.append( pointTuple )

		# find two nearest points from OpenCV's world coordinate frame
		# init vector's magnitude storage
		vectorEndPointToVectorMagnitudeDict = dict()

		# loop through each point in approximated polygon
		for point in  approximatedPolygon1DList:

			# calculate vector magnitude and store it
			vectorEndPointToVectorMagnitudeDict[ point ] = calculateVectorMagnitude( ( 0, 0 ), point )

		# sort each point based of its vector's lenght
		sortedVectorEndPointToVectorMagnitudeDict = dict( sorted( vectorEndPointToVectorMagnitudeDict.items(), key=lambda x: x[ 1 ] ) )

		# get the most nearest point from OpenCV's world coordinate frame
		theNearestPointFromWorldCoordinateTuple = list( sortedVectorEndPointToVectorMagnitudeDict.keys() )[ 0 ]

		# get the secord nearest point from OpenCV's world coordinate frame
		theSecordNearestPointFromWorldCoordinateTuple = list( sortedVectorEndPointToVectorMagnitudeDict.keys() )[ 1 ]

		# find start point and end point of vector which is the same direction as contour's x-axis frame
		# in this case, end point is the point with largest x component
		if theNearestPointFromWorldCoordinateTuple[ 1 ] > theSecordNearestPointFromWorldCoordinateTuple[ 1 ]:
			vectorWithTheSameDirectionAsYAxisTuple = ( theNearestPointFromWorldCoordinateTuple, theSecordNearestPointFromWorldCoordinateTuple )
		vectorWithTheSameDirectionAsYAxisTuple = ( theSecordNearestPointFromWorldCoordinateTuple, theNearestPointFromWorldCoordinateTuple )
	
		# calculate directional vector
		directionalVectorXAxisComponentInt = vectorWithTheSameDirectionAsYAxisTuple[ 1 ][ 0 ] - vectorWithTheSameDirectionAsYAxisTuple[ 0 ][ 0 ]
		directionalVectorYAxisComponentInt = vectorWithTheSameDirectionAsYAxisTuple[ 1 ][ 1 ] - vectorWithTheSameDirectionAsYAxisTuple[ 0 ][ 1 ]
		directionalVectorTuple = ( directionalVectorXAxisComponentInt, directionalVectorYAxisComponentInt )

		# calculate directional vector magnitude
		directionalVectorMagnitude = calculateVectorMagnitude( ( 0, 0 ), directionalVectorTuple )
		
		# normalize vector to obtain a unit vector
		normalizedDirectionalVectorXAxisComponentFloat = directionalVectorXAxisComponentInt / directionalVectorMagnitude
		normalizedDirectionalVectorYAxisComponentFloat = directionalVectorYAxisComponentInt / directionalVectorMagnitude

		# include size to the directional vector 
		normalizedDirectionalVectorWithSpecificSizeTuple = ( self.vectorLengthInt * normalizedDirectionalVectorXAxisComponentFloat,
															self.vectorLengthInt * normalizedDirectionalVectorYAxisComponentFloat )
		
		# find end point of contour's coordinate frame x-axis vector
		self.xAxisEndPointTuple = ( self.centerPointTuple[ 0 ] + normalizedDirectionalVectorWithSpecificSizeTuple[ 0 ], 
			     					self.centerPointTuple[ 1 ] + + normalizedDirectionalVectorWithSpecificSizeTuple[ 0 ])




		





In [4]:
# create image plotter object
imagePlotter = ImagePlotter( 7, 7, 2 )

# reading image
originalImage = cv2.imread( 'field_image_from_manual.png' )

# add original image to figure
imagePlotter.addImageToPlot( originalImage, 'original image' )

# converting image into grayscale image
grayScaleImage = cv2.cvtColor( originalImage, cv2.COLOR_BGR2GRAY )

# add gray scale image to figure 
imagePlotter.addImageToPlot( grayScaleImage, 'gray scale image' )

# convert to binary image
_, binaryImage = cv2.threshold( grayScaleImage, 127, 255, cv2.THRESH_BINARY )

# add binary image to figure 
imagePlotter.addImageToPlot( binaryImage, 'binary image' )

# finding contours from binary image
contourList, _ = cv2.findContours( binaryImage, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE )

# init contour storage
contourStorageObj = ContourStorage( contourList )

# filter contour only in area range
contourObjInAreaRangeList = contourStorageObj.filterContoursOnlyInAreaRange( 1000, 300000 )

# copy original image to draw contour in area range
originalImageForDrawContourInAreaRange = originalImage.copy()

# loop through contour in area range
for contourObj in contourObjInAreaRangeList:
	
	# using drawContours() function
	cv2.drawContours( originalImageForDrawContourInAreaRange, [ contourObj.contour ], 0, (0, 0, 255), 3 )

# add original image with contour in area range to figure 
imagePlotter.addImageToPlot( originalImageForDrawContourInAreaRange, 'contour in area range' )

# copy original image to draw circle contour
originalImageForDrawCircleContour = originalImage.copy()

# copy original image to draw triangle contour
originalImageForDrawTriangleContour = originalImage.copy()

# copy original image to draw square contour
originalImageForDrawSquareContour = originalImage.copy()

# classify contour object by its shape
contourStorageObj.classifyContourObjByShape()

print( contourStorageObj.circleContourObjList )
print( contourStorageObj.triangleContourObjList )
print( contourStorageObj.squareContourObjList )

# loop through each circle contour
for circleContourObj in contourStorageObj.circleContourObjList:

	# draw contour 
	cv2.drawContours( originalImageForDrawCircleContour, [ circleContourObj.contour ], 0, (0, 0, 255), 3 )

# add original image with circle contour 
imagePlotter.addImageToPlot( originalImageForDrawCircleContour, 'circle contour' )

# loop through each circle contour
for triangleContourObj in contourStorageObj.triangleContourObjList:

	# draw contour 
	cv2.drawContours( originalImageForDrawTriangleContour, [ triangleContourObj.contour ], 0, (0, 0, 255), 3 )

# add original image with triangle contour 
imagePlotter.addImageToPlot( originalImageForDrawTriangleContour, 'triangle contour' )

# loop through each circle contour
for squareContourObj in contourStorageObj.squareContourObjList:

	# draw contour 
	cv2.drawContours( originalImageForDrawSquareContour, [ squareContourObj.contour ], 0, (0, 0, 255), 3 )

# add original image with square contour 
imagePlotter.addImageToPlot( originalImageForDrawSquareContour, 'square contour' )

# copy original image to draw outer most circle contour
originalImageForDrawOuterMostCircleContour = originalImage.copy()

# copy original image to draw outer most triangle contour
originalImageForDrawOuterMostTriangleContour = originalImage.copy()

# copy original image to draw outer most square contour
originalImageForDrawOuterMostSquareContour = originalImage.copy()

# filter only outer most circle contour object
contourStorageObj.outerMostCircleContourObjList = contourStorageObj.filterOnlyOuterMostContourObj( contourStorageObj.circleContourObjList, 4 )

# loop through each outer most circle contour object
for circleContourObj in contourStorageObj.outerMostCircleContourObjList:

	# draw contour 
	cv2.drawContours( originalImageForDrawOuterMostCircleContour, [ circleContourObj.contour ], 0, ( 0, 0, 255 ), 3 )

	# draw center point of contour
	cv2.circle( originalImageForDrawOuterMostCircleContour, circleContourObj.centerPointTuple, 7, ( 255, 255, 255 ), -1)

	# draw x-axis line of coordinate frame
	cv2.line( originalImageForDrawOuterMostCircleContour, circleContourObj.centerPointTuple, circleContourObj.xAxisEndPointTuple, ( 0, 0, 255 ), 3)

	# draw y-axis line of coordinate frame
	cv2.line( originalImageForDrawOuterMostCircleContour, circleContourObj.centerPointTuple, circleContourObj.yAxisEndPointTuple, ( 0, 255, 0 ), 3)

# add original image with outer most circle contour to figure
imagePlotter.addImageToPlot( originalImageForDrawOuterMostCircleContour, 'outer most circle contour' )

# filter only outer most triangle contour object
contourStorageObj.outerMostTriangleContourObjList = contourStorageObj.filterOnlyOuterMostContourObj( contourStorageObj.triangleContourObjList, 5 )

# loop through each outer most triangle contour object
for triangleContourObj in contourStorageObj.outerMostTriangleContourObjList:

	# draw contour 
	cv2.drawContours( originalImageForDrawOuterMostTriangleContour, [ triangleContourObj.contour ], 0, ( 0, 0, 255 ), 3 )

	# draw center point of contour
	cv2.circle( originalImageForDrawOuterMostTriangleContour, triangleContourObj.centerPointTuple, 7, ( 255, 255, 255 ), -1)

# add original image with outer most triangle contour to figure
imagePlotter.addImageToPlot( originalImageForDrawOuterMostTriangleContour, 'outer most triangle contour' )

# filter only outer most square contour object
contourStorageObj.outerMostSquareContourObjList = contourStorageObj.filterOnlyOuterMostContourObj( contourStorageObj.squareContourObjList, 4 )

# loop through each outer most square contour object
for squareContourObj in contourStorageObj.outerMostSquareContourObjList:

	# draw contour 
	cv2.drawContours( originalImageForDrawOuterMostSquareContour, [ squareContourObj.contour ], 0, ( 0, 0, 255 ), 3 )

	# draw center point of contour
	cv2.circle( originalImageForDrawOuterMostSquareContour, squareContourObj.centerPointTuple, 7, ( 255, 255, 255 ), -1)

# add original image with outer most square contour to figure
imagePlotter.addImageToPlot( originalImageForDrawOuterMostSquareContour, 'outer most square contour' )

# show figure
imagePlotter.plotAndShowAllImage()

[<__main__.Contour object at 0x7fe7f012d030>, <__main__.Contour object at 0x7fe7f012dcf0>, <__main__.Contour object at 0x7fe7f012dd50>, <__main__.Contour object at 0x7fe7f012c2b0>, <__main__.Contour object at 0x7fe7f012c370>, <__main__.Contour object at 0x7fe7f012f7c0>]
[<__main__.Contour object at 0x7fe7f014d870>, <__main__.Contour object at 0x7fe7f012d3c0>, <__main__.Contour object at 0x7fe7f012c8b0>, <__main__.Contour object at 0x7fe7f012dc00>, <__main__.Contour object at 0x7fe7f012d900>, <__main__.Contour object at 0x7fe7f012efb0>]
[<__main__.Contour object at 0x7fe7f012d0f0>, <__main__.Contour object at 0x7fe7f012fb20>, <__main__.Contour object at 0x7fe7f012edd0>, <__main__.Contour object at 0x7fe7f012eda0>, <__main__.Contour object at 0x7fe7f012ed40>, <__main__.Contour object at 0x7fe7f012ee00>, <__main__.Contour object at 0x7fe7f012cca0>, <__main__.Contour object at 0x7fe7f012e860>]


error: OpenCV(4.7.0) :-1: error: (-5:Bad argument) in function 'line'
> Overload resolution failed:
>  - Can't parse 'pt2'. Sequence item with index 0 has a wrong type
>  - Can't parse 'pt2'. Sequence item with index 0 has a wrong type


In [ ]:
# def filterContoursOnlyInAreaRange( contourList, minArea=0, maxArea=0 ):
# 	''' - get only contour in area range

# 		ARGS:
# 			- contourList ( list )
# 			- minArea ( int )
# 			- maxArea ( int )

# 		RETURN:
# 			- contourListInAreaRange ( list )
# 	'''

# 	# contour in area range storage
# 	contourListInAreaRange = []

# 	# loop through each contour
# 	for contour in contourList:

# 		# calculate contour area
# 		contourArea = cv2.contourArea( contour )

# 		# this contour is in range
# 		if minArea <= contourArea <= maxArea:
			
# 			# store it
# 			contourListInAreaRange.append( contour )
# 	return contourListInAreaRange

# def approximatePolygonOfContour( contour, epsilonPercent ):
# 	''' - approximate polygon of contour to be a rougher shape

# 		- mainly used for shape analysis

# 		- usually set epsilon parameter to 1 - 5 % of contour's perimeter
# 	'''

# 	# calculate contour perimeter
# 	contourPerimeter = cv2.arcLength( contour, True )

# 	# approximate curve of contour
# 	approximatedPointList = cv2.approxPolyDP( contour,  epsilonPercent / 100 * contourPerimeter, True )
# 	return approximatedPointList

# def isCircleContour( contour, epsilonPercent, errorPercent ):
# 	''' - check if this is a circle contour

# 		- this contour in circle if:
# 			- approximate point is more than5
# 			- bounding box has square aspect ratio
# 	'''

# 	# calculate approximation curve of contour
# 	approximatedPointList = approximatePolygonOfContour( contour, epsilonPercent  )

# 	# contour has polygon approximation point more than 5
# 	if len( approximatedPointList ) > 5:

# 		# calculate approximation polygon bounding box
# 		( xPosition, yPosition, boundingBoxWidth, boundingBoxHeight ) = cv2.boundingRect( approximatedPointList )
		
# 		# calculate aspect ratio of bounding box
# 		boundingBoxAspectRatio = boundingBoxWidth / boundingBoxHeight

# 		# calculate acceptable error
# 		acceptableError = errorPercent / 100 

# 		# this bounding box is square
# 		if 1 - acceptableError <= boundingBoxAspectRatio <= 1 + acceptableError:
			
# 			# this contour is circle
# 			return True
		
# 	# this contour is not circle
# 	return False

# def isTriangleContour( contour, epsilonPercent ):
# 	''' - check if this is a triangle shape

# 		- this contour is triangle if:
# 			- it approximation curve has 3 points 
# 	''' 

# 	# calculate approximation curve of contour
# 	approximatedPointList = approximatePolygonOfContour( contour, epsilonPercent  )

# 	# this is triangle shape contour
# 	if len( approximatedPointList ) == 3:
# 		return True
	
# 	# this is not a triangle contour
# 	return False

# def isSquareContour( contour, epsilonPercent, errorPercent ):
# 	''' - check if this contour is square shape

# 		- this contour is square contour if:
# 			- its approximation curve has 4 points
# 			- its bounding box has an aspect ration close to 1
# 	'''

# 	# calculate approximation curve of contour
# 	approximatedPointList = approximatePolygonOfContour( contour, epsilonPercent  )

# 	# contour has polygon approximation point equal to 4
# 	if len( approximatedPointList ) == 4:

# 		# calculate approximation polygon bounding box
# 		( xPosition, yPosition, boundingBoxWidth, boundingBoxHeight ) = cv2.boundingRect( approximatedPointList )
		
# 		# calculate aspect ratio of bounding box
# 		boundingBoxAspectRatio = boundingBoxWidth / boundingBoxHeight

# 		# calculate acceptable error
# 		acceptableError = errorPercent / 100 

# 		# this bounding box is square
# 		if 1 - acceptableError <= boundingBoxAspectRatio <= 1 + acceptableError:
			
# 			# this is square contour
# 			return True
		
# 	# this contour is not square
# 	return False

# def isOuterMostContour( contour, otherContourList ):
# 	''' - check to see if this contour is NOT inside any of the other contour
# 	'''

# 	# approximate polygon of contour
# 	approximatedPolygonList = approximatePolygonOfContour( contour, 4 )

# 	# loop through each point in contour
# 	for otherContour in otherContourList:

# 		# loop each point in approximated polygon
# 		for point in approximatedPolygonList:

# 			pointTuple = ( int( point[ 0 ][ 0 ] ), int( point[ 0 ][ 1 ] ) )

# 			# point is inside other approximated polygon
# 			if cv2.pointPolygonTest( otherContour, pointTuple, False ) > 0:
				
# 				# this is NOT the outer most contour
# 				return False

# 	# this is the outer most contour
# 	return True